In [0]:
from dotenv import load_dotenv
import os
import random
from typing import Literal, TypedDict
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated
from databricks_langchain import ChatDatabricks
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import MessagesState

# load_dotenv()


# import mlflow
# mlflow.langchain.autolog()





llm = ChatDatabricks(endpoint = "databricks-gpt-oss-120b")

def call_llm(state: MessagesState):

    system_msg = {"role": "system", "content": "Reply only with plain text. No formatting."}
    all_msgs = [system_msg] + state["messages"]

    return {"messages": [llm.invoke(all_msgs)]}


builder = StateGraph(MessagesState)
builder.add_node("call_llm", call_llm)


builder.add_edge(START, "call_llm")
builder.add_edge("call_llm", END)

graph = builder.compile()

messages = graph.invoke({"messages": [HumanMessage("Tell me more about Databricks ?")]})

In [0]:
message = messages['messages'][-1]

for part in message.content:
    if part.get("type") == "text":
       ai_message = part.get("text", "")

print(ai_message)